# Processing the final dataset



In [2]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"

In [3]:
# region of interest is Amazonia
# roi = ee.FeatureCollection(f"{data_folder}/biomes_br").first().geometry()
# For specific biomes, the feature "CD_Bioma" is:
# 0 = Amazonia
# 1 = Caatinga
# 2 = Cerrado
# 3 = Mata Atlantica
# 4 = Pampa
# 5 = Pantanal

In [5]:
age_agbd = ee.Image(f"{data_folder}/age_agbd")
categorical = ee.Image(f"{data_folder}/categorical")
sur_cover = ee.Image(f"{data_folder}/sur_cover")
one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")
land_use = ee.Image(f"{data_folder}/land_use")
land_use_5_years = ee.Image(f"{data_folder}/land_use_5_years")
land_use_10_years = ee.Image(f"{data_folder}/land_use_10_years")
land_use_15_years = ee.Image(f"{data_folder}/land_use_15_years")

zuquim_soil = ee.Image(f"{data_folder}/raw/zuquim_amazon_soil_SB")
cwd = ee.Image(f"{data_folder}/raw/cwd_chave").float()
yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
yearly_SI = ee.Image(f"{data_folder}/yearly_SI")


# unified_data = age_agbd.addBands([land_use_10_years, cwd]).updateMask(land_use_10_years.select('last_LU')).updateMask(one_hectare_mask)
unified_data = (
    age_agbd.addBands(
        [land_use_5_years, cwd, categorical, sur_cover, yearly_mean_prec, yearly_SI]
    )
    .updateMask(land_use_5_years.select("last_LU"))
    .updateMask(one_hectare_mask)
    .addBands(age_agbd.mask().toInt().rename("mask"))
)

In [ ]:
data_folder = "projects/amazon-forest-regrowth/assets"

land_use = ee.Image(f"{data_folder}/land_use")
map = geemap.Map()

map.addLayer(land_use, {})
map

In [9]:
data_folder = "projects/amazon-forest-regrowth/assets"

# # using stratified sample in order to get only the points that have a value for age
unified_data_fc = age_agbd.select('age').updateMask(land_use_5_years.select("last_LU")).sample(
    region = age_agbd.geometry(), geometries = True
)

# unified_data_fc = ages.sample(
#     region = ages.geometry(), numPixels = 1e8
# )



# import matplotlib.pyplot as plt

# # 19, 24, 29
# histogram_ee = age_agbd.select('age').updateMask(land_use_5_years.select("last_LU")).reduceRegion(
#     geometry = age_agbd.select('age').updateMask(land_use_5_years.select("last_LU")).geometry(), reducer = ee.Reducer.histogram(), maxPixels=1e13
# )

# hist = histogram_ee.getInfo()

# # list(hist.keys())
# # Get the bin centers and bin counts
# bin_centers = hist['age']['bucketMeans']
# bin_counts = hist['age']['histogram']

# # Plot the histogram
# plt.bar(bin_centers, bin_counts, width=1)
# plt.xlabel('Age')
# plt.ylabel('Count')
# plt.title('Histogram of Ages with 5 years of land use history or less')
# plt.show()


# # Export an ee.FeatureCollection as an Earth Engine asset.
# task = ee.batch.Export.table.toDrive(
#     collection=unified_data_fc, description="unified_ages"
# )
# task.start()

EEException: Computed value is too large.